## 第2週 2日目

私たちの最初のエージェント・フレームワーク・プロジェクト!!

途方もなく簡単な何かのために自分自身を準備してください。

（SDR（セールス・ディベロップメント・レプレゼンタティブ）エージェントの構築に向けた、）  
コールド・セールス・アウト・リーチ・メールを生成するためのシンプルなエージェントシステムを構築。  
（コールド・セールス・メールとは、見込み顧客にファースト・コンタクトする営業メール）

1. エージェントワークフロー
1. 関数を呼び出すためのツールの使用
1. ツールとハンドオフを介したエージェントコラボレーション

## 開始する前に - いくつかのセットアップ：

SendGridをご覧ください: https://sendgrid.com/

（SendGridは、メールを送信するためのTwilio社のサービス）

SendGridが問題を抱えている場合は、[community_contributions/2_lab2_with_resend_email](https://github.com/ed-donner/agents/blob/main/2_openai/community_contributions/2_lab2_with_resend_email.ipynb)で「resendメール」を使用した代替実装を参照してください

（SendGridは、個人利用NGなので個人メアドはハネられる♨）

アカウントを設定してください - 無料です！ （少なくとも、私にとっては、今）。

アカウントを作成したら、クリックしてください。

設定（左サイドバー）>> APIキー>> APIキーを作成（右上のボタン）

キーをクリップボードにコピーし、.envファイルに新しい行を追加します。

`SENDGRID_API_KEY=xxxx`

また、SendGrid内で、次のように移動します。

設定（左サイドバー）>>送信者認証>>「単一の送信者を確認する」  
また、自分のメールアドレスが実際のメールアドレスであることを確認してください。これにより、SendGridがメールを送信できるようにします。

In [1]:
#!pip install resend

In [2]:
import os
import asyncio
from typing import Dict

from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent

import resend # import sendgrid
#from sendgrid.helpers.mail import Mail, Email, To, Content

In [3]:
load_dotenv(override=True)

True

In [4]:
# 電子メールがあなたのために働いていることを確認しましょう

def send_test_email():
    #sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))    
    resend.api_key=os.getenv("RESEND_API_KEY")

    #from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
    #to_email = To("ed.donner@gmail.com")  # Change to your recipient
    #content = Content("text/plain", "This is an important test email")
    #mail = Mail(from_email, to_email, "Test email", content).get()
    #response = sg.client.mail.send.post(request_body=mail)
    
    response = resend.Emails.send({
        "from": "onboarding@resend.dev",
        "to": "nishi_74322014@ksj.biglobe.ne.jp",
        "subject": "Hello World",  "html": "<p>Congrats on sending your <strong>first email</strong>!</p>"
    })

    # print(response.status_code)
    print(response)

send_test_email()

{'id': 'e0bc4c10-6e33-4196-9762-6cec163f2cfa'}


### メールを受け取りましたか？

202を手に入れたら、行ってもいいです！

#### 証明書エラー

certificate_verify_failedエラーが発生した場合、学生の Chris S と Oleksandr kが提案を持っています。  
最初にこれを実行します： `!uv pip install --upgrade certifi`  
次に、これを実行します：
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

#### その他のエラーまたは電子メールなし

他の問題がある場合は、SendGridダッシュボードでAPIキーと確認済みの送信者のメールアドレスを確認する必要があります

または、[community_contributions/2_lab2_with_resend_email](https://github.com/ed-donner/agents/blob/main/2_openai/community_contributions/2_lab2_with_resend_email.ipynb)で「resendメール」を使用して代替実装を使用します

（または - 以下の電子メール送信コードをPushover コールに置き換えること、または単純にフラット・ファイルに書き込むものをいつでも置き換えることができます）

## ステップ1：エージェントワークフロー

### instruction

In [5]:
#instruction1「あなたはComplAIで営業担当者として働いています。
# ComplAIは、AIを活用したSOC2コンプライアンスの確保と監査対策のためのSaaSツールを提供する企業です。
# あなたは「プロフェッショナルで真摯な」コールドメールを作成しています。」
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

# instructions2「あなたはComplAIで営業担当者として働いています。
# ComplAIは、AIを活用したSOC2コンプライアンスの確保と監査対策のためのSaaSツールを提供する企業です。
# あなたは「返信を期待できる、機知に富んだ魅力的な」コールドメールを作成しています。」
instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

# instructions3「あなたはComplAIで営業担当者として働いています。
# ComplAIは、AIを活用したSOC2コンプライアンスの確保と監査対策のためのSaaSツールを提供する企業です。
# あなたは「簡潔で要点を押さえた」コールドメールを作成しています。」
instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

### Agent

#### Agentを生成して

In [6]:
# プロフェッショナルな販売代理店
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)
# 魅力的な販売代理店
sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)
# 多忙な販売代理店
sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

#### Agentを実行

##### Agentを１つ実行

In [7]:
# AIエージェントを非同期で実行し、その出力をストリーミング表示する処理

# .run_streamed() は 「ストリーミング実行」を意味する。
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")

# ストリーム・イベントを非同期で取得
async for event in result.stream_events():
    # モデルのテキスト出力イベントで差分が生成されたら
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        # 受け取ったテキスト差分を改行せずバッファリングせず即時出力。
        print(event.data.delta, end="", flush=True)

Subject: Simplifying Your SOC 2 Compliance Process with AI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m with ComplAI, where we specialize in streamlining SOC 2 compliance and audit preparation through our innovative SaaS platform.

Navigating the complexities of SOC 2 can be a significant challenge for many organizations, often consuming valuable time and resources. Our AI-powered tool simplifies this process by automating documentation, providing real-time compliance monitoring, and delivering actionable insights tailored to your unique needs.

Some key benefits of our platform include:
- **Automated Documentation**: Reduce manual efforts and minimize errors in compliance documentation.
- **Real-Time Monitoring**: Stay ahead of compliance requirements with continuous oversight.
- **Audit Preparation Made Easy**: Generate audit-ready reports with just a few clicks.

I’d love to schedule a brief call to discuss how ComplAI can help your or

##### Agentを並列実行

In [8]:
# コールド・セールス・メールを書く
message = "Write a cold sales email"

# OpenAI API PF → Dashboard → Logs → Traces の 
with trace("Parallel cold emails"): # 並行したコールド・メール
    # 非同期IO的な意味で並列実行
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

# 非同期なので、ストリーミング出力は無理
outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Subject: Simplify Your SOC 2 Compliance with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. I'm reaching out to introduce you to ComplAI, an innovative SaaS tool designed to streamline SOC 2 compliance and audit preparation.

As you know, maintaining compliance can be a complex and time-consuming process. ComplAI harnesses the power of AI to automate key compliance tasks, providing you with a clear, organized approach to managing your documentation and audit readiness.

Here are a few ways ComplAI can benefit your organization:

- **Automated Documentation**: Generate and manage your compliance documents effortlessly.
- **Real-Time Monitoring**: Stay informed with ongoing compliance assessments and alerts, reducing the risk of last-minute surprises.
- **Efficient Preparation**: Simplify the audit process, saving your team valuable time for other critical projects.

I would love the opportunity to discuss how ComplAI can add value to your compliance efforts and sup

##### Agentを並列＋直列実行
オーケストレーション＆ワークフローみたいな。

In [9]:
# セールス・ピッカー：提示された選択肢の中から、最適なコールド・セールス・メールを選択してください。
# 自分が顧客だと想像し、最も返信しそうなメールを選んでください。説明は不要です。選択したメールのみで返信してください。",
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [10]:
# コールド・セールス・メールを書く
message = "Write a cold sales email"

# OpenAI API PF → Dashboard → Logs → Traces の 
with trace("Selection from sales people"): # 営業担当者からの選択
    # 非同期IO的な意味で並列実行
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

    # results 中の各 result から最終的な出力（テキスト）だけを取り出し、リスト化。
    outputs = [result.final_output for result in results]

    # 複数のメール案を 1つのテキストにまとめ比較しやすい形に。
    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    # sales_pickerがメールを選択
    best = await Runner.run(sales_picker, emails)

    # sales_pickerが選択したメールを出力
    print(f"Best sales email:\n{best.final_output}")


Best sales email:
Subject: Is Your Compliance Strategy Ready for Its Close-Up? 🎬

Hey [First Name],

Hope this email finds you as well as your company’s SOC2 audit prep finds you stressed! 😅 

Let’s face it, preparing for compliance can feel like a never-ending game of “Where’s Waldo” – only Waldo is a mountain of documents, and the prize is a stress-induced headache! 🎢 

At ComplAI, we believe that ensuring SOC2 compliance shouldn't feel like climbing Everest in flip-flops. Our AI-powered tool turns the chaos of audits into a smooth, breezy walk in the park (with a picnic, if you’re lucky). 🍉🥪

Imagine this:
- Effortlessly centralize your compliance efforts.
- Easily access audit-ready documentation at your fingertips.
- Save time (and sanity) while doing it all!

If you're ready to trade in those flip-flops for some solid hiking boots, let’s chat! I promise, no pressure—just a friendly conversation about how we can help make SOC2 compliance your new favorite pastime.

Looking forward

今すぐ行ってトレースをチェックしてください：

https://platform.openai.com/traces

## ステップ2、3：ツールとエージェントの相互作用

ここでツールを追加します。

JSON の定型コードと、if ロジックを含む `handle_tool_calls()` 関数を思い出してください。

そのすべてのボイラープレートJSONを覚えていますか？機能をデコレータ「@function_tool」で包むだけです。

In [11]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    
    #sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    resend.api_key=os.getenv("RESEND_API_KEY")
    
    #from_email = Email("ed@edwarddonner.com")  # 検証済みの送信者に変更します
    #to_email = To("ed.donner@gmail.com")  # 受信者に変更します
    #content = Content("text/plain", body)
    #mail = Mail(from_email, to_email, "Sales email", content).get()
    #sg.client.mail.send.post(request_body=mail)

    response = resend.Emails.send({
        "from": "onboarding@resend.dev",
        "to": "nishi_74322014@ksj.biglobe.ne.jp",
        "subject": "Sales email",
        "text": body
    })

    return {"status": "success"}

{'id': '2fce68d0-e6bf-4dfd-acc3-de789ad6a0f3'}


### @function_toolでJSONに自動的変換

In [12]:
# 見てみましょう（JSON）
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f5556357380>, strict_json_schema=True, is_enabled=True)

### またエージェントをツールに変換できる

In [13]:
# エージェントをツールに変換
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")

# 見てみましょう（JSON）
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f554b458c20>, strict_json_schema=True, is_enabled=True)

### すべてのツールを一緒にまとめられる

- 3つの電子メールライティングエージェントのそれぞれのツール
- そして、私たちの関数が電子メールを送信するためのツール

In [14]:
# コールド・セールス・メールを書く
description = "Write a cold sales email"

# エージェントをツールに変換
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

# ツールにまとめる。
tools = [tool1, tool2, tool3, send_email]

# 見てみましょう（JSON）
tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f5554d79ee0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f5556c74cc0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'req

### セールスマネージャー - 計画エージェント

In [15]:
# 学生Guillermo Fのおかげで指示の改善

# あなたはComplAIの営業マネージャーです。あなたの目標は、sales_agentツールを使って、最適なコールド・セールス・メールを1通見つけることです。
# 
# 以下の手順を慎重に実行してください。
# 1. 下書きの作成：3つのsales_agentツールすべてを使って、3種類のメール下書きを作成します。3つの下書きがすべて完成するまで、先に進めないでください。
# 2. 評価と選択：下書きを確認し、最も効果的だと判断した上で、最適なメールを1通選びます。
# 3. send_emailツールを使用して、最適なメール（そして最適なメールのみ）をユーザーに送信します。
# 
# 重要なルール：
# - 下書きの作成には、sales_agentツールを使用する必要があります。自分で作成しないでください。
# - send_emailツールを使用して送信するメールは1通のみです。複数のメールは送信しないでください。

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""

# セールスマネージャー - 計画エージェント
sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

#「CEO様」宛て「コールド・セールス・メール」を送信
message = "Send a cold sales email addressed to 'Dear CEO'"

# OpenAI API PF → Dashboard → Logs → Traces の 
with trace("Sales manager"):
    #「CEO様」宛て「コールド・セールス・メール」を送信
    result = await Runner.run(sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">えっ、メールが届いてない？</h2>
            <span style="color:#ff7800;">問題と解決策を説明してくださった学生のChris S.さんに心より感謝いたします。前のセルを実行してもメールが届かない場合は、以下の点を確認してください。: <br/>
            （１）まず、スパムフォルダを確認してください！何人かの学生が、メールがスパムフォルダに入っていることに気づいていないようです！（２）次に、print(result)を実行して、SSLに関するエラーが発生していないか確認してください。SSLエラーが発生している場合は、こちらの <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">ネットワークのヒント</a> を確認し、次のセルの注記をご覧ください。（３）また、OpenAIのトレースを確認し、SendGridのウェブサイトで調査して手がかりを探してみてください。何かお手伝いできることがあればお知らせください！
            </span>
        </td>
    </tr>
</table>

### そして、Windows 11で学生のOleksandrから電子メールを送信するためのもう1つの提案：

証明書SSLエラーが発生している場合は、  
ターミナルでこれを実行します： `uv pip install --upgrade certifi`

次に、このコードを実行します。
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Oleksandrありがとう！

### トレースを確認することを忘れないでください

https://platform.openai.com/traces

そして、あなたのメールをチェックしてください!!

## ハンドオフ：エージェントがエージェントに委任（コントロールごと渡す）

- ハンドオフとエージェントとしてのエージェントは似ています
- どちらの場合も、エージェントは別のエージェントと協力できます
- ツールを使用すると、コントロールが戻る
- ハンドオフを使用すると、コントロールが通過します

In [16]:
# コールド・セールス・メールの件名を書くことができます。メッセージの内容に応じて、返信を期待できる件名を書く必要があります。
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

# テキストメールの本文を HTML メールの本文に変換。マークダウンが含まれている可能性のあるテキストメールの本文を、シンプルで明確、かつ魅力的なレイアウトとデザインの HTML メールの本文に変換する。
html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

# 件名考案エージェントをツール化
subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

# MD → HTML変換エージェントをツール化
html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [17]:
# テキストメールからHTMLに改修されている。
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    
    #sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    resend.api_key=os.getenv("RESEND_API_KEY")
    
    #from_email = Email("ed@edwarddonner.com")  # 検証済みの送信者に変更します
    #to_email = To("ed.donner@gmail.com")  # 受信者に変更します
    #content = Content("text/html", html_body)
    #mail = Mail(from_email, to_email, subject, content).get()
    #sg.client.mail.send.post(request_body=mail)
    
    response = resend.Emails.send({
        "from": "onboarding@resend.dev",
        "to": "nishi_74322014@ksj.biglobe.ne.jp",
        "subject": subject,
        "html": html_body
    })

    return {"status": "success"}

# メール送信のテスト
send_test_email()

{'id': '5de23cb6-7d30-4235-a12c-912caf7ebe89'}


In [18]:
# ツールにまとめる。
tools = [subject_tool, html_tool, send_html_email]

# 見てみましょう（JSON）
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f5549bbd120>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f5549bbd260>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to all sale

In [19]:
# あなたはメールのフォーマッター兼送信者です。送信するメールの本文を受け取ります。
# まず、subject_writer ツールを使用してメールの件名を作成し、次に html_converter ツールを使用して本文を HTML に変換します。
# 最後に、send_html_email ツールを使用して、件名と HTML 本文を含むメールを送信します。
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

# メール関係のハンドリングを（メールをHTMLに変換して送信）するエージェント
emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

### これで、3つのツールと1つのハンドオフがあります

In [20]:
# （エージェントを）ツールにまとめる。
tools = [tool1, tool2, tool3]
print(tools)

# （メール関係のハンドリングは）ハンドオフする。
handoffs = [emailer_agent]
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f5554d79ee0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f5556c74cc0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'requi

In [21]:
# 学生Guillermo Fのおかげで指示の改善
# メール送信の所がツール使用からハンドオフニ変更されている。

# あなたはComplAIの営業マネージャーです。あなたの目標は、sales_agentツールを使って、最適なコールド・セールス・メールを1通見つけることです。
# 
# 以下の手順を慎重に実行してください。
# 1. 下書きの作成：3つのsales_agentツールすべてを使って、3種類のメール下書きを作成します。3つの下書きがすべて完成するまで、先に進めないでください。
# 2. 評価と選択：下書きを確認し、最も効果的だと判断した上で、最適なメールを1通選びます。最初の結果に満足できない場合は、ツールを複数回使用できます。
# 3. 送信のためのハンドオフ：採用されたメールの下書きのみをEmail Managerエージェントに渡します。Email Managerがフォーマットと送信を行います。
# 
# 重要なルール：
# - 下書きの作成には、sales_agentツールを使用する必要があります。自分で作成しないでください。
# - Email Managerに引き継ぐメールは1通のみで、複数回は引き継がないでください。

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""

# Sales Manager
sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    # （エージェントを）ツールにまとめる。
    tools=tools,
    # （メール関係のハンドリングは）ハンドオフする。
    handoffs=handoffs,
    model="gpt-4o-mini")

# アリスから「CEO様」宛て「コールド・セールス・メール」を送信
message = "Send out a cold sales email addressed to Dear CEO from Alice"

# OpenAI API PF → Dashboard → Logs → Traces の 
with trace("Automated SDR"):
    # アリスから「CEO様」宛て「コールド・セールス・メール」を送信
    result = await Runner.run(sales_manager, message)

### トレースを確認することを忘れないでください

https://platform.openai.com/traces

そして、あなたのメールをチェックしてください!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">演習</h2>
            <span style="color:#ff7800;">
                ここで使用されているAgenticのデザインパターンを特定できますか？<br/>
                Anthropicの定義において、Agenticの「ワークフロー」から「エージェント」へと変化させた1行は何ですか？<br/>
                ツールとエージェントをもっと追加してみましょう！メーリングリストへの送信のための差し込み印刷を処理するツールも作成できます。<br/>
                難しい課題：ユーザーがメールに返信したときにSendGridがコールバックWebhookを呼び出し、SDRが応答して会話を継続させる方法を調べてください。これには「バイブコーディング」が必要になるかもしれません😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">商業的影響</h2>
            <span style="color:#00bfff;">
                これはセールスオートメーションにすぐに応用できますが、より一般的には、会話とツールを介したあらゆるビジネスプロセスのエンドツーエンドの自動化に適用できます。このようなエージェントソリューションを日々の業務にどのように適用できるか考えてみてください。
            </span>
        </td>
    </tr>
</table>

## 余分なメモ：

Googleはエージェント開発キット（ADK）をリリースしました。このコースの他のフレームワークの牽引力はまだありませんが、注目を集めています。

OpenAI Agents SDKに非常によく似ていることに注意するのは興味深いことです。プレビューを提供するには、ADKのサンプルコードのピークを次に示します。

```python
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

まあ、それはおなじみに見えます！

また、学生はADKを使用するCommunity_Contributionsにカスタマーケアエージェントに貢献しました（どれ？？）